In [1]:
%pip install tensorflow_similarity

/home/michal/.local/lib/python3.8/site-packages/cryptography/hazmat/backends/openssl/x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(
     |████████████████████████████████| 206 kB 4.5 MB/s eta 0:00:01
     |████████████████████████████████| 4.7 MB 33.6 MB/s eta 0:00:01
     |████████████████████████████████| 88 kB 15.1 MB/s eta 0:00:01
     |████████████████████████████████| 13.4 MB 109 kB/s eta 0:00:011
     |████████████████████████████████| 52 kB 3.7 MB/s  eta 0:00:01
     |████████████████████████████████| 140 kB 39.1 MB/s eta 0:00:01
     |████████████████████████████████| 110 kB 48.2 MB/s eta 0:00:01
     |████████████████████████████████| 408 kB 122.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 139.9 MB/s eta 0:00:01
     |████████████████████████████████| 188 kB 27.0 MB/s eta 0:00:01
     |████████████████████████████████| 217 kB 47.7 MB/s eta 0:00:

In [2]:
%pip install tensorflow

/home/michal/.local/lib/python3.8/site-packages/cryptography/hazmat/backends/openssl/x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(
     |████████████████████████████████| 588.3 MB 65 kB/s  eta 0:00:016  | 299.4 MB 1.6 MB/s eta 0:03:01��██            | 366.9 MB 11.5 MB/s eta 0:00:20��██▏     | 481.1 MB 20.5 MB/s eta 0:00:06 | 484.8 MB 826 kB/s eta 0:02:06 | 493.4 MB 821 kB/s eta 0:01:560 MB 891 kB/s eta 0:00:39
     |████████████████████████████████| 4.5 MB 22.5 MB/s eta 0:00:01
     |████████████████████████████████| 6.0 MB 10.3 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 1.1 MB/s  eta 0:00:01
     |████████████████████████████████| 2.4 MB 1.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 63.8 MB/s eta 0:00:01  | 655 kB 63.8 MB/s eta 0:00:01
     |████████████████████████████████| 439 kB 6.7 MB/s eta 0:00:01
     |████████████████████████

In [2]:
import tensorflow_similarity as tfsim
import numpy as np
import tensorflow as tf
import pandas as pd
import sklearn
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

## Config

In [5]:
BATCH_SIZE = 64
NUM_EPOCHS = 20
SHAPE = (2000,)
TRAINING_SAMPLE_SIZE = 150000

## Data

df = pd.DataFrame()

In [6]:
def get_model():
    inputs = tf.keras.layers.Input(shape=SHAPE)
    x = tf.keras.layers.BatchNormalization()(inputs)
    x = tf.keras.layers.Dense(256)(x)
    x = tf.keras.layers.Dense(128)(x)
    outputs = tfsim.layers.MetricEmbedding(64)(x)
    return tfsim.models.SimilarityModel(inputs, outputs)

model = get_model()
model.summary()

Model: "similarity_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2000)]            0         
                                                                 
 batch_normalization (BatchN  (None, 2000)             8000      
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 256)               512256    
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 metric_embedding (MetricEmb  (None, 64)               8256      
 edding)                                                         
                                                                 
Total params: 561,408
Trainable params: 557,408
No

2022-11-27 01:29:58.281704: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :~/School/gurobi592/linux64/lib
2022-11-27 01:29:58.281726: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-27 01:29:58.281740: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (michal-Prestige-14-A11SCX): /proc/driver/nvidia/version does not exist
2022-11-27 01:29:58.281926: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate comp

In [ ]:
def get_compiled_model(lr=0.00005, distance = "cosine"):  # @param {type:"number"}
    model = get_model()
    loss = tfsim.losses.TripletLoss(distance=distance)
    model.compile(optimizer=tf.keras.optimizers.Adam(lr), loss=loss)
    return model

In [ ]:
sampler = tfsim.samplers.MultiShotMemorySampler(df_train['slice'], df_train['id'], classes_per_batch=10, examples_per_class_per_batch=4)

In [ ]:
model = get_compiled_model(lr=0.0001)
hist = model.fit(sampler, 
                     steps_per_epoch=1000,
                     epochs=40,
                     verbose=2,)


## Evaluate

In [ ]:
def append_emebddings_to_df(df, model):
    rv = []
    for image_batch in np.array_split(df.slice, BATCH_SIZE):
        images = np.vstack(image_batch)
        dataset = tf.data.Dataset.from_tensor_slices(images).batch(BATCH_SIZE)
        preds = model.predict(dataset)
        rv.extend(preds)
    df['embeddings'] = rv
    return df

In [ ]:
df_train_pandas = append_emebddings_to_df(df_train, model)

In [ ]:
plt.rcParams["figure.figsize"] = (20,13)
df_pcad = visualize_pca_for_patients(df_train_pandas, n_patients=20, n_samples_per_patient=40)